# FateZ Explain 

This notebook demonstrate how to utilize explanatory methods of FateZ models.

In [1]:
import os
import sys
import torch
from torch.utils.data import DataLoader
import fatez.test as test
import fatez.model as model
import fatez.tool.JSON as JSON
import fatez.process as process
import fatez.process.explainer as explainer
import fatez.process.fine_tuner as fine_tuner
import fatez.process.pre_trainer as pre_trainer
from pkg_resources import resource_filename

# Ignoring warnings because of using LazyLinear
import warnings
warnings.filterwarnings('ignore')

print('Done')

Done


### Build model and make some fake data first.

In [2]:
# Parameters
params = {
    'n_sample': 10,       # Fake samples to make
    'batch_size': 1,      # Batch size
}

# Load built-in config file
config = JSON.decode(resource_filename(
        __name__, '../../fatez/data/config/gat_bert_config.json'
    )
)

factory_kwargs = {'device': 'cuda', 'dtype': torch.float32,}

# Generate Fake data
faker = test.Faker(model_config = config, **params)
train_dataloader = faker.make_data_loader()

print('Done')

Done


### Now we perform pre-training with no label.


Here trainer's $train\_adj$ is set to False, and the model is NOT reconstructing the adjacency matrices, etc.

In [3]:
trainer = pre_trainer.Set(config, factory_kwargs)
report = trainer.train(train_dataloader, report_batch = True)
print(report)

        Loss
0   1.565720
1   2.900557
2   2.166636
3   1.762677
4   2.366010
5   2.911416
6   2.797966
7   1.742560
8   2.400287
9   1.946299
10  2.256013


In the case of pre-training with reconstructing adjacency matrices as well.

In [4]:
config['pre_trainer']['train_adj'] = True
trainer = pre_trainer.Set(config, factory_kwargs)
report = trainer.train(train_dataloader, report_batch = True)
print(report)

        Loss
0   2.821428
1   1.732386
2   2.861434
3   2.155074
4   2.810074
5   2.665386
6   1.765009
7   2.106514
8   1.699396
9   2.549504
10  2.316621


### Then, we can go for fine tuning part with class labels.

In [5]:
tuner = fine_tuner.Set(config, factory_kwargs, prev_model = trainer.model)
report = tuner.train(train_dataloader, report_batch = True, quiet = True)
print(report)

        Loss  ACC
0   0.707717  0.0
1   0.656518  1.0
2   0.832635  0.0
3   0.608765  1.0
4   0.518559  1.0
5   0.511055  1.0
6   0.960859  0.0
7   0.875309  0.0
8   0.917681  0.0
9   0.931165  0.0
10  0.752026  0.4


### To explain Fine Tuning model in general. 

Note: to make overall conclusion on the contribution of a specific gene, we would need to sum up importance values for every feature dimension (RNA-count, peaks)

In [6]:
# Suppress debug print outs
suppressor = process.Quiet_Mode()
suppressor.on()

# Get background data
background_data = DataLoader(train_dataloader.dataset, batch_size = params['n_sample'])
background_data = [a for a, _ in background_data][0]
background_data = [a.to(factory_kwargs['device']) for a in background_data]
explain = explainer.Gradient(tuner.model, background_data)
print('Suppress here?')
# vars can be used to estimate how accurate the explanation would be: lower the better
for input_data, _ in train_dataloader:
    input_data = [i.to(factory_kwargs['device']).to_dense() for i in input_data]
    gene_shap_values, vars = explain.shap_values(input_data, return_variances = True)
    break
suppressor.off()
    
    
print(f'Explaining {len(gene_shap_values)} classes.')

# Having 2 inputs
assert len(gene_shap_values[0]) == 2

# Only the feat mat explanation should be working
print(f'Each class has explain in shape of {gene_shap_values[0][0].shape}.')

# The adj mat explanation should NOT be working since lacking gradient
# print(gene_shap_values[0][1].shape)

Explaining 2 classes.
Each class has explain in shape of (1, 10, 2).


### To explain the BERT part for analyzing importances of TFs only.

Note: similarly, we would want to sum up values across embed dimensions.

In [17]:
# We also should accumulate gat_out for every trained input.
explain = explainer.Gradient(
    tuner.model.bert_model, 
    tuner.model.get_gat_output(background_data[0], background_data[1])
)

# Here I just make 1 gat_out for example
regulon_shap_values, vars = explain.shap_values(
    tuner.model.get_gat_output(input_data[0], input_data[1]), 
    return_variances=True
)

print(f'Explaining {len(regulon_shap_values)} classes.')
print(f'Each class has explain in shape of {regulon_shap_values[0].shape}.')

Explaining 2 classes.
Each class has explain in shape of (1, 4, 4).


### To explain the GAT for analyzing GRP importances.

The grp_explain here is purely based on the GAT attention weights.

In [18]:
grp_explain = tuner.model.gat.explain(input_data[0][0], input_data[1][0])
print(grp_explain.shape)

# Or we can feed in matrices with ones to extract attention weights.
grp_explain = tuner.model.gat.explain(
    torch.ones_like(input_data[0][0]),
    torch.ones_like(input_data[1][0])
)
print(grp_explain.shape)

torch.Size([4, 10])
torch.Size([4, 10])


Utilizing the importance values of each gene or TF regulon inferred from the calculated shapley values above would be sufficent to obtain importances of each GRP.

In [19]:
# Sum up shapley values of each features for every node (gene or TF).
regulon_importance_values = regulon_shap_values[0].sum(2)
gene_importance_values = gene_shap_values[0][0].sum(2)
print(regulon_importance_values.shape)
print(gene_importance_values.shape)
grp_explain = grp_explain.to(factory_kwargs['device'])
grp_importance = torch.matmul(torch.Tensor(regulon_importance_values[0]).to(factory_kwargs['device']), grp_explain)
grp_importance = torch.matmul(grp_explain, torch.Tensor(gene_importance_values[0]).to(factory_kwargs['device']))

(1, 4)
(1, 10)
